# Auto encoder based

In [13]:
import scipy.io as sio
import yaml
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

from torch import nn
from typing import Dict, List

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, ToTensor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.metrics import accuracy_score

import utils as u
import RPNetRFextractor as rp

from sklearn.decomposition import PCA
from sklearn.svm import SVC, LinearSVC


from sklearn.model_selection import train_test_split

configuration = yaml.safe_load(open('config.yaml'))
data_path= configuration["data"]
parameters = configuration["parameters"]


In [14]:

# Auto-encoders


class Encoder(nn.Module):
    def __init__(self,n_in: int,
                  n_out: int,
                  n_hidden: List[int] ):
        
        super.__init__()
        self.n_in = n_in
        self.n_out = n_out
        self.n_hidden= n_hidden

        in_list= [n_in] + n_hidden
        out_list= n_hidden + n_out

        self.encoder= nn.Sequential(
            *[nn.Sequential(nn.Linear(x, y), nn.ReLU()) for x,y in zip(in_list, out_list)]
        )

    def forward(self, x: torch.Tensor):
        return self.encoder(x)
    
class Decoder(nn.Module):
    def __init__(self,n_in: int,
                  n_out: int,
                  n_hidden: List[int]):
        super.__init__(self)
        self.n_in = n_in
        self.n_out = n_out
        self.n_hidden= n_hidden

        in_list= [n_in] + n_hidden
        out_list= n_hidden + n_out

        self.decoder= nn.Sequential(
            *[nn.Sequential(nn.ReLU(), nn.Linear(x, y)) for x,y in zip(in_list, out_list)]
        )

    def forward(self, x: torch.Tensor):
        return self.decoder(x)
    
class AE(nn.Module):
    def __init__(self, n_in: int, 
                 bottleneck: int, 
                 n_hidden_encoder: List[int], 
                 n_hidden_decoder: List[int] = None):
        super.__init__(self)
        self.n_in = n_in
        self.n_hidden_encoder=n_hidden_encoder
        self.bottleneck = bottleneck
        if n_hidden_decoder is None:
            # basecase : symetrical
            self.n_hidden_decoder= n_hidden_encoder[::-1]
        else:
            self.n_hidden_decoder= n_hidden_decoder

        self.encoder = Encoder(n_in, bottleneck, n_hidden_encoder)
        self.decoder = Decoder(bottleneck, n_in, self.n_hidden_decoder)

    def forward(self, x: torch.Tensor):
        return(self.encoder(self.encoder(x)))
    
    def encode(self, x: torch.Tensor):
        return(self.encoder(x))
    
    def decode(self, y: torch.Tensor):
        return(self.decode(y))


In [5]:
nn.Sequential(*[nn.Linear(10, 15), nn.ReLU(), nn.Linear(15, 17)])

Sequential(
  (0): Linear(in_features=10, out_features=15, bias=True)
  (1): ReLU()
  (2): Linear(in_features=15, out_features=17, bias=True)
)

# Tesing the 1D CNN approach

# 2D CNN approach

# GANS